In [1]:
import pandas as pd
import os
import re
import json
from owlready2 import *
from getpass import getpass
import collections
from xlrd import open_workbook
from openpyxl import load_workbook

In [15]:
def prepara_lista_bacias(df_depbr, df_dmrr):
    '''
    Função que exporta arquivo json com sinônimos de bacias, agrupados pelo código da bacia ANP.
    Entradas: 
        df_depbr: DataFrame com a tabela BACIAS_SEDIMENTARES do DEPBR
        df_dmrr: DataFrame com o Information Link Dimensão Bacia do DMRR
    Saída: json com os sinônimos para cada bacia sedimentar.
    '''
    columns_mapper = {'Bacia Código': 'Bacia Código',
                      'Bacia Nome': 'BACIA_SINONIMO',
                      'Bacia Nome Ambiente': 'Bacia Nome Ambiente',
                      'Bacia Nome Bacia Sedimentar': 'BASE_NM_BACIA',
                      'Bacia Sigla': 'BACIA_SIGLA',
                      'Bacia Sigla Bacia Sedimentar': 'BASE_SG_BACIA'}
    df_dmrr = df_dmrr.rename(columns = columns_mapper)
    
    df = pd.merge(left=df_depbr, right=df_dmrr, how='outer', on=["BASE_NM_BACIA", "BASE_SG_BACIA"])
    
    sinonimos_bacias = {}
    #LISTA DE BACIAS SEDIMENTARES: COLUNA 'BASE_NM_BACIA' DA TABELA BACIAS_SEDIMENTARES (DEPBR)
    #CÓDIGO IDENTIFICADOR DE BACIAS SEDIMENTARES: 'BASE_CD_BACIA' DA TABELA BACIAS_SEDIMENTARES (DEPBR)
    for i in range(len(df)):
        bacia_sedimentar = df.loc[i]['BASE_NM_BACIA']
        if bacia_sedimentar != 'NÃO ATRIBUÍDO':
            codigo_bacia = int(df.loc[i]['BASE_CD_BACIA'])

            if codigo_bacia not in sinonimos_bacias.keys():
                sinonimos_bacias[codigo_bacia] = [bacia_sedimentar,'BACIA DE '+bacia_sedimentar]

        #LISTA DE SINÔNIMOS: Bacia Nome DO IL DIMENSÃO BACIA (DMRR)
            if isinstance(df.loc[i]['BACIA_SINONIMO'], str):
                sinonimo_bacia = df.loc[i]['BACIA_SINONIMO']
                if sinonimo_bacia not in sinonimos_bacias[codigo_bacia]:
                    sinonimos_bacias[codigo_bacia].append(sinonimo_bacia)
    #EXCEÇÕES PARA BACIA DE CAMPOS E SANTOS (COMUMENTE REFERIDAS COMO BC E BS)
    sinonimos_bacias[281].append('BC')
    sinonimos_bacias[316].append('BC')
    sinonimos_bacias[15] = ["MADRE DE DIOS", "BACIA DE MADRE DE DIOS"]
    sinonimos_bacias[55] = ["AMAPÁ", "BACIA DE AMAPÁ"]
    sinonimos_bacias[60] = ["BRAGANCA VIZEU", "BRAGANCA VIZEU-ILHA NOVA", "BACIA DE BRAGANCA VIZEU"]
    sinonimos_bacias[95] = ["MALHADO VERMELHO", "BACIA DE MALHADO VERMELHO"]
    sinonimos_bacias[110] = ["IGUATU", "BACIA DE IGUATU"]
    sinonimos_bacias[125] = ["LIMA CAMPOS", "BACIA DE LIMA CAMPOS"]
    sinonimos_bacias[130] = ["ICÓ", "BACIA DE ICÓ"]
    sinonimos_bacias[135] = ["POMBAL", "BACIA DE POMBAL"]
    sinonimos_bacias[140] = ["SOUZA", "BACIA DE SOUZA"]
    sinonimos_bacias[145] = ["TRIUNFO", "BACIA DE TRIUNFO"]
    sinonimos_bacias[155] = ["BARRO", "BACIA DE BARRO"]
    sinonimos_bacias[160] = ["CEDRO", "BACIA DE CEDRO"]
    sinonimos_bacias[165] = ["SERRA DO INÁCIO", "BACIA DE SERRA DO INÁCIO"]
    sinonimos_bacias[170] = ["SÃO JOSÉ DO BELMONTE", "BACIA DE SÃO JOSÉ DO BELMONTE"]
    sinonimos_bacias[180] = ["MIRANDIBA", "BACIA DE MIRANDIBA"]
    sinonimos_bacias[175] = ["BOM NOME", "BACIA DE BOM NOME"]
    sinonimos_bacias[185] = ["TUPANACI", "BACIA DE TUPANACI"]
    sinonimos_bacias[190] = ["BETÂNIA", "BACIA DE BETÂNIA"]
    sinonimos_bacias[195] = ["AFOGADOS DA INGAZEIRA", "BACIA DE AFOGADOS DA INGAZEIRA"]
    sinonimos_bacias[205] = ["ITABERABA", "BACIA DE ITABERABA"]
    sinonimos_bacias[285] = ["ITABORAÍ", "BACIA DE ITABORAÍ"]
    sinonimos_bacias[290] = ["RESENDE", "BACIA DE RESENDE"]
    sinonimos_bacias[295] = ["TAUBATÉ", "BACIA DE TAUBATÉ"]
    sinonimos_bacias[305] = ["SÃO PAULO", "BACIA DE SÃO PAULO"]
    sinonimos_bacias[310] = ["CURITIBA", "BACIA DE CURITIBA"]
    sinonimos_bacias[330] = ["ÁGUA BONITA", "BACIA DE ÁGUA BONITA"]

    return sinonimos_bacias

In [16]:
def define_anexados(df):
    '''
    Cria dicionário com campos de destino e respecticva lista de campos anexados
    Entrada: DataFrame com tabela de campos exportada do SIRR
    Saída: dicionário de campos anexados:
        *chaves são os nomes completos dos campos de destino (str);
        *itens são os nomes abreviados e completos dos campos anexados (lista) ao campo de destino correspondente.
    '''
    mapper_anexados = {}
    for i in range(len(df)):
        if isinstance(df.loc[i]['Campos anexados'], str):
            sinonimos = []
            anexados =  df.loc[i]['Campos anexados'].strip('[').strip(']').split(', ')
            for anexado in anexados:
                sinonimos.append(list(df[df['Nome (abreviado)']==anexado]['Nome (completo)'])[0])
                
                #PARA VERIFICAR SE O CAMPO ANEXADO FOI POSTERIORMENTE ANEXADO A OUTRO CAMPO
                #ATENÇÃO: NÃO RESOLVE CASOS DE MÚLTIPLAS ANEXAÇÕES 
                if isinstance(list(df[df['Nome (abreviado)']==anexado]['Campos anexados'])[0], str):
                    sub_anexados = list(df[df['Nome (abreviado)']==anexado]['Campos anexados'])[0].strip('[').strip(']').split(', ')
                    for sub_anexado in sub_anexados:
                        sinonimos.append(list(df[df['Nome (abreviado)']==sub_anexado]['Nome (completo)'])[0])
                    sinonimos = sinonimos+sub_anexados
            anexados = anexados+sinonimos
            mapper_anexados[df.loc[i]['Nome (completo)']] = list(set(anexados))
    return mapper_anexados

def define_status(df):
    '''
    Cria dicionário com códigos de status do campo
    Entrada: DataFrame com tabela de campos exportada do SIRR
    Saída: dicionário de código de campos:
        *chaves são os códigos (int);
        *itens são a descrição do código correspondente (str)
    '''
    mapper_codigo_status = {}
    for codigo in list(df['Status (cod)'].unique()):
        mapper_codigo_status[codigo] = df[df['Status (cod)']==codigo]['Status (desc)'].unique()[0]
    return mapper_codigo_status

In [17]:
def prepara_lista_campos(df, mapper_anexados):   
    '''
    Função que exporta arquivo json com sinônimos de campos, agrupados pelo código do campo.
    Entradas: 
        df: DataFrame com a tabela CAMPO do DEPBR
        mapper_anexados: com campos de destino e respecticva lista de campos anexados (gerada pela função define_anexados)
    Saída: json com os sinônimos para cada campo.
    '''
    sinonimos_campos = {}
    #LISTA DE CAMPOS: COLUNA 'CAMP_NM_CAMPO' DA TABELA CAMPO (DEPBR)
    #NÃO INCLUSOS: CAMPOS COM STATUS ANEXADO (12), BLOCO EXPLORATÓRIO-DEVOLVIDO PARA ANP (2), BLOCO EXPLORATÓRIO (1)
    #CÓDIGO IDENTIFICADOR DE CAMPOS: 'CAMP_CD_CAMPO' DA TABELA CAMPO (DEPBR)
    nao_inclusos = [1, 2, 12]
    for i in range(len(df)):
        if df.loc[i]['STCA_CD_CAMPO'] not in nao_inclusos:
            
            #LISTA DE SINÔNIMOS: COLUNAS COM NOME ABREVIADO (CAMP_NM_ABREV) E NOME COMPLETO (CAMP_NM_CAMPO) DA TABELA CAMPO
            sinonimo_campo = []
            sinonimo_campo.append(df.loc[i]['CAMP_NM_CAMPO'])
            sinonimo_campo.append(df.loc[i]['CAMP_NM_ABREV'])
            
            #LISTA DE SINÔNIMOS: CAMPOS ANEXADOS (LISTA ORIUNDA DO SIRR)
            if df.loc[i]['CAMP_NM_CAMPO'] in mapper_anexados.keys():
                sinonimo_campo = sinonimo_campo+mapper_anexados[df.loc[i]['CAMP_NM_CAMPO']]

            codigo_campo = int(df.loc[i]['CAMP_CD_CAMPO'])
            if codigo_campo not in sinonimos_campos.keys():
                sinonimos_campos[codigo_campo] = list(set(sinonimo_campo))
    return sinonimos_campos

In [18]:
def prepara_lista_campo_bacia(df_campo, df_bacia):
    '''
    Função que exporta arquivo json com relação entre campo e bacia, agrupados pelo código do campo.
    Entradas: 
        df_campo: DataFrame com a tabela de campos exportada do SIRR
        df_bacia: DataFrame com o Information Link Dimensão Bacia do DMRR
    Saída: json com o nome da bacia sedimentar para cada código de campo.
    '''
    campo_bacia = {}
    
    for i in range(len(df_campo)):
        codigo_campo = int(df_campo.loc[i]['Código Petrobras'])
        if codigo_campo not in campo_bacia.keys():
            bacia = df_campo.loc[i]['Bacia']
            if bacia in list(df_bacia['Bacia Sigla']):
                campo_bacia[codigo_campo] = list(df_bacia[df_bacia['Bacia Sigla']==bacia]['Bacia Nome Bacia Sedimentar'])[0]
            else:
                print(df_campo.loc[i]['Nome (completo)'])
    return campo_bacia

In [40]:
def prepara_lista_est_text_por(df):   
    '''
    Função que exporta arquivo json com estruturas deposicionais, textura de rocha e tipo de porosidade,
    agrupadas por sua sigla.
    Entradas: 
        df: DataFrame com a tabela de Estruturas Deposicionais, Texturas ou Tipos de Porosidade do estudo do dicionário
    '''
    sinonimos_est_text_por = {}
    
    n=int(df['Código'].max())

    df = df.fillna(0)
    for i in range(len(df)):
        codigo_entidade = int(df.loc[i]['Código'])
        if codigo_entidade == 0:
            codigo_entidade = n
            n+=1
        nome_entidade = df.loc[i]['Nome']
        if codigo_entidade not in sinonimos_est_text_por.keys():
            sinonimos_est_text_por[codigo_entidade] = nome_entidade
    return sinonimos_est_text_por

In [20]:
file_bacia = load_workbook(filename='resources/bacia/bacia.xlsx', data_only=True)

In [21]:
file_bacia.sheetnames

['depbr', 'dmrr']

In [22]:
df_depbr_bacias_sedimentares = pd.read_excel('resources/bacia/bacia.xlsx', sheet_name = 'depbr')
df_dmrr_bacias = pd.read_excel('resources/bacia/bacia.xlsx', sheet_name = 'dmrr')

In [23]:
sinonimos_bacias = prepara_lista_bacias(df_depbr_bacias_sedimentares, df_dmrr_bacias)

In [24]:
with open('resources/bacia/bacias.json', 'w') as f:
    json.dump(sinonimos_bacias, f)

In [25]:
file_campo = load_workbook(filename='resources/campo/campo.xlsx', data_only=True)

In [26]:
file_campo.sheetnames

['ppid', 'depbr', 'sirr', 'dicionario']

In [27]:
df_sirr_campos = pd.read_excel('resources/campo/campo.xlsx', sheet_name = 'sirr')
df_depbr_campos = pd.read_excel('resources/campo/campo.xlsx', sheet_name = 'depbr')

In [28]:
mapper_anexados = define_anexados(df_sirr_campos)

In [29]:
sinonimos_campos = prepara_lista_campos(df_depbr_campos, mapper_anexados)

In [30]:
with open('resources/campo/campos.json', 'w') as f:
    json.dump(sinonimos_campos, f)

In [31]:
campo_localizado_bacia = prepara_lista_campo_bacia(df_sirr_campos, df_dmrr_bacias)

In [32]:
with open('resources/relacoes/campo_bacia.json', 'w') as f:
    json.dump(campo_localizado_bacia, f)

In [33]:
file_est_text_por = load_workbook(filename='resources/estrutura_textura_porosidade/estrutura_textura_porosidade.xlsx', 
                                  data_only=True)

In [34]:
file_est_text_por.sheetnames

['Estrutura - BR',
 'Estrutura - Outros',
 'Textura - BR',
 'Textura - Outros',
 'Porosidade - BR',
 'listas',
 'SQL']

In [35]:
df_estrutura = pd.read_excel('resources/estrutura_textura_porosidade/estrutura_textura_porosidade.xlsx', 
                             sheet_name = 'Estrutura - BR')
df_textura = pd.read_excel('resources/estrutura_textura_porosidade/estrutura_textura_porosidade.xlsx', 
                           sheet_name = 'Textura - BR')
df_porosidade = pd.read_excel('resources/estrutura_textura_porosidade/estrutura_textura_porosidade.xlsx', 
                              sheet_name = 'Porosidade - BR')

In [41]:
sinonimos_estrutura = prepara_lista_est_text_por(df_estrutura)
sinonimos_textura = prepara_lista_est_text_por(df_textura)
sinonimos_porosidade = prepara_lista_est_text_por(df_porosidade)

In [42]:
with open('resources/estrutura_textura_porosidade/estrutura.json', 'w') as f:
    json.dump(sinonimos_estrutura, f)

with open('resources/estrutura_textura_porosidade/textura.json', 'w') as f:
    json.dump(sinonimos_textura, f)

with open('resources/estrutura_textura_porosidade/porosidade.json', 'w') as f:
    json.dump(sinonimos_porosidade, f)